# Deploy the REST API

•	Go to the *Environment* tab, then click on the wheel icon and enable *Allow incoming connections*. The API will be accessible through the indicated tunnelling link.  
•	Run the notebook.



In [1]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path


REDIS_HOST = 'redis-13184.c250.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 13184
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'rD5R7TcvcTfjV47VPdDXgXUlksrUPbhg'

# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

# endpoint /status
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

# endpoint /todos
class TodoList(object):
    exposed = True

    def GET(self, *path, **query):
        # print(query)
        keys = redis_client.keys('todo:*')
        print(keys)
        items = []

        completed = query.get('completed', None)
        if completed is not None:
            completed = bool(completed)

        message = query.get('message', None)

        for key in keys:
            key = key.decode()
            item = redis_client.json().get(key)
            uid = key.removeprefix('todo:')
            item['id'] = uid

            if completed is not None and message is not None:
                if completed == item['completed'] and message in item['message']:
                    items.append(item)
            elif completed is not None:
                if completed == item['completed']:
                    items.append(item)
            elif message is not None:
                if message in item['message']:
                    items.append(item)
            else:
                items.append(item)

        response = json.dumps(items)

        return response

    def POST(self, *path, **query):
        uid = uuid.uuid4()
        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        todo_data = {
            'message': body_dict['message'],
            'completed': False,
        }
        redis_client.json().set(f'todo:{uid}', Path.root_path(), todo_data)
        
        return

# endpoint /todo/{id}
class TodoDetail(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing id')
        uid = path[0]

        item = redis_client.json().get(f'todo:{path[0]}')

        if item is None:
             raise cherrypy.HTTPError(404, '404 Not Found')
        
        item['id'] = uid

        response = json.dumps(item)
        
        return response

    def PUT(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing id')
        uid = path[0]

        item = redis_client.json().get(f'todo:{path[0]}')

        if item is None:
             raise cherrypy.HTTPError(404, '404 Not Found')
        
        body = cherrypy.request.body.read()
        body_dict = json.loads(body.decode())

        redis_client.json().set(f'todo:{uid}', Path.root_path(), body_dict)
        
        return

    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing id')
        uid = path[0]
        
        found = redis_client.delete(f'todo:{uid}')

        if found == 0:
            raise cherrypy.HTTPError(404, '404 Not Found')

        return

if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(TodoList(), '/todos', conf)
    cherrypy.tree.mount(TodoDetail(), '/todo', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[14/Dec/2023:15:10:21] ENGINE Bus STARTING
[14/Dec/2023:15:10:21] ENGINE Started monitor thread 'Autoreloader'.
[14/Dec/2023:15:10:21] ENGINE Serving on http://0.0.0.0:8080
[14/Dec/2023:15:10:21] ENGINE Bus STARTED
Redis Connected: True
172.3.27.152 - - [14/Dec/2023:15:10:25] "GET /status HTTP/1.1" 200 20 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 OPR/105.0.0.0"
[14/Dec/2023:15:10:28] ENGINE Keyboard Interrupt: shutting down bus
[14/Dec/2023:15:10:28] ENGINE Bus STOPPING
[14/Dec/2023:15:10:28] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 8080)) shut down
[14/Dec/2023:15:10:28] ENGINE Stopped thread 'Autoreloader'.
[14/Dec/2023:15:10:28] ENGINE Bus STOPPED
[14/Dec/2023:15:10:28] ENGINE Bus EXITING
[14/Dec/2023:15:10:28] ENGINE Bus EXITED
[14/Dec/2023:15:10:28] ENGINE Waiting for child threads to terminate...


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f1fd91f-a434-4542-983d-3ce5ae14ac33' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>